In [ ]:
!pip install neuralforecast utilsforecast datasetsforecast

In [ ]:
import pandas as pd

from datasetsforecast.m3 import M3
from datasetsforecast.m4 import M4

from utilsforecast.losses import mae, smape
from utilsforecast.evaluation import evaluate

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS

In [ ]:
def get_dataset(name):
    if name == 'M3-yearly':
        Y_df, *_ = M3.load("./data", "Yearly")
        horizon = 6
        freq = 'Y'
    elif name == 'M3-quarterly':
        Y_df, *_ = M3.load("./data", "Quarterly")
        horizon = 8
        freq = 'Q'
    elif name == 'M3-monthly':
        Y_df, *_ = M3.load("./data", "Monthly")
        horizon = 18
        freq = 'M'
    elif name == 'M4-yearly':
        Y_df, *_ = M4.load("./data", "Yearly")
        Y_df['ds'] = Y_df['ds'].astype(int)
        horizon = 6
        freq = 1
    elif name == 'M4-quarterly':
        Y_df, *_ = M4.load("./data", "Quarterly")
        Y_df['ds'] = Y_df['ds'].astype(int)
        horizon = 8
        freq = 1
    elif name == 'M4-monthly':
        Y_df, *_ = M4.load("./data", "Monthly")
        Y_df['ds'] = Y_df['ds'].astype(int)
        horizon = 18
        freq = 1
    elif name == 'M4-weekly':
        Y_df, *_ = M4.load("./data", "Weekly")
        Y_df['ds'] = Y_df['ds'].astype(int)
        horizon = 13
        freq = 1
    elif name == 'M4-daily':
        Y_df, *_ = M4.load("./data", "Daily")
        Y_df['ds'] = Y_df['ds'].astype(int)
        horizon = 14
        freq = 1
    elif name == 'M4-hourly':
        Y_df, *_ = M4.load("./data", "Hourly")
        Y_df['ds'] = Y_df['ds'].astype(int)
        horizon = 48
        freq = 1

    return Y_df, horizon, freq

In [ ]:
MODEL_NAMES = ['NBEATS_identity',
               'NBEATS_poly2', 'NBEATS_poly3', 'NBEATS_poly4',
               'NBEATS_pwl2', 'NBEATS_pwl3', 'NBEATS_pwl4',
               'NBEATS_changepoint2', 'NBEATS_changepoint3', 'NBEATS_changepoint4']
for dataset in ['M4-hourly', 'M4-weekly', 'M4-quarterly', 'M4-monthly', 'M4-daily', 'M4-yearly', 'M3-yearly', 'M3-quarterly', 'M3-monthly']:

    Y_df, horizon, freq = get_dataset(dataset)
    test_df = Y_df.groupby('unique_id').tail(horizon)
    train_df = Y_df.drop(test_df.index).reset_index(drop=True)
    input_size = 2*horizon

    nbeats_identity = NBEATS(
        input_size=input_size,
        h=horizon,
        stack_types = ['identity', 'identity', 'identity'],
        early_stop_patience_steps=3,
        max_steps=1000,
        alias="NBEATS_identity"
    )

    nbeats_poly2 = NBEATS(
        input_size=input_size,
        h=horizon,
        n_basis=2,
        basis='polynomial',
        stack_types = ["identity", "trend", "seasonality"],
        early_stop_patience_steps=3,
        max_steps=1000,
        alias="NBEATS_poly2"
    )
    nbeats_poly3 = NBEATS(
        input_size=input_size,
        h=horizon,
        n_basis=3,
        basis='polynomial',
        stack_types = ["identity", "trend", "seasonality"],
        early_stop_patience_steps=3,
        max_steps=1000,
        alias="NBEATS_poly3"
    )
    nbeats_poly4 = NBEATS(
        input_size=input_size,
        h=horizon,
        n_basis=4,
        basis='polynomial',
        stack_types = ["identity", "trend", "seasonality"],
        early_stop_patience_steps=3,
        max_steps=1000,
        alias="NBEATS_poly4"
    )

    nbeats_pwl2 = NBEATS(
        input_size=input_size,
        h=horizon,
        n_basis=2,
        basis='piecewise_linear',
        stack_types = ["identity", "trend", "seasonality"],
        early_stop_patience_steps=3,
        max_steps=1000,
        alias="NBEATS_pwl2"
    )
    nbeats_pwl3 = NBEATS(
        input_size=input_size,
        h=horizon,
        n_basis=3,
        basis='piecewise_linear',
        stack_types = ["identity", "trend", "seasonality"],
        early_stop_patience_steps=3,
        max_steps=1000,
        alias="NBEATS_pwl3"
    )
    nbeats_pwl4 = NBEATS(
        input_size=input_size,
        h=horizon,
        n_basis=4,
        basis='piecewise_linear',
        stack_types = ["identity", "trend", "seasonality"],
        early_stop_patience_steps=3,
        max_steps=1000,
        alias="NBEATS_pwl4"
    )

    nbeats_changepoint2 = NBEATS(
        input_size=input_size,
        h=horizon,
        n_basis=2,
        basis='changepoint',
        stack_types = ["identity", "trend", "seasonality"],
        early_stop_patience_steps=3,
        max_steps=1000,
        alias="NBEATS_changepoint2"
    )
    nbeats_changepoint3 = NBEATS(
        input_size=input_size,
        h=horizon,
        n_basis=3,
        basis='changepoint',
        stack_types = ["identity", "trend", "seasonality"],
        early_stop_patience_steps=3,
        max_steps=1000,
        alias="NBEATS_changepoint3"
    )
    nbeats_changepoint4 = NBEATS(
        input_size=input_size,
        h=horizon,
        n_basis=4,
        basis='changepoint',
        stack_types = ["identity", "trend", "seasonality"],
        early_stop_patience_steps=3,
        max_steps=1000,
        alias="NBEATS_changepoint4"
    )

    models = [nbeats_identity, nbeats_changepoint2, nbeats_changepoint3, nbeats_changepoint4,
              nbeats_poly2, nbeats_poly3, nbeats_poly4, nbeats_pwl2, nbeats_pwl3, nbeats_pwl4]

    nf = NeuralForecast(models=models, freq=freq)
    nf.fit(train_df, val_size=horizon)
    preds = nf.predict()
    preds = preds.reset_index(drop=True)
    test_df = pd.merge(test_df, preds, 'left', ['ds', 'unique_id'])

    evaluation = evaluate(
        test_df,
        metrics=[mae, smape],
        models=MODEL_NAMES,
        target_col="y",
    )
    evaluation = evaluation.drop(['unique_id'], axis=1).groupby('metric').mean().reset_index()
    print(dataset)
    print(evaluation)